In [ ]:
!pip install fiftyone
!pip install ipython

In [33]:
def yolo_bbox2segment(im_dir, save_dir=None, sam_model="sam2_l.pt"):
    """
    Converts existing object detection dataset (bounding boxes) to segmentation dataset or oriented bounding box (OBB)
    in YOLO format. Generates segmentation data using SAM auto-annotator as needed.

    Args:
        im_dir (str | Path): Path to image directory to convert.
        save_dir (str | Path): Path to save the generated labels, labels will be saved
            into `labels-segment` in the same directory level of `im_dir` if save_dir is None. Default: None.
        sam_model (str): Segmentation model to use for intermediate segmentation data; optional.

    """

    # NOTE: add placeholder to pass class index check
    dataset = YOLODataset(im_dir, data=dict(names=list(range(1000))))

    if len(dataset.labels[0]["segments"]) > 0:  # if it's segment data
        LOGGER.info("Segmentation labels detected, no need to generate new ones!")
        return
    LOGGER.info("Detection labels detected, generating segment labels by SAM model!")
    sam_model = SAM(sam_model)
    
    #process YOLO labels and generate segmentation masks using the SAM model
    for l in tqdm(dataset.labels, total=len(dataset.labels), desc="Generating segment labels"):
        h, w = l["shape"]
        boxes = l["bboxes"]
        if len(boxes) == 0:  # skip empty labels
            continue
        boxes[:, [0, 2]] *= w
        boxes[:, [1, 3]] *= h
        im = cv2.imread(l["im_file"])
        sam_results = sam_model(im, bboxes=xywh2xyxy(boxes), verbose=False, save=False)
        l["segments"] = sam_results[0].masks.xyn

    save_dir = Path(save_dir) if save_dir else Path(im_dir).parent / "labels-segment"
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Saves segmentation masks and class labels to text files for each image
    for l in dataset.labels:
        texts = []
        lb_name = Path(l["im_file"]).with_suffix(".txt").name
        txt_file = save_dir / lb_name
        cls = l["cls"]
        for i, s in enumerate(l["segments"]):
            line = (int(cls[i]), *s.reshape(-1))
            texts.append(("%g " * len(line)).rstrip() % line)
        if texts:
            with open(txt_file, "a") as f:
                f.writelines(text + "\n" for text in texts)

    LOGGER.info(f"Generated segment labels saved in {save_dir}")

In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v8.3.0/sam2_l.pt

In [ ]:
train_dataset = "/home/nata-brain/Documents/proj/cv-train-fn/train"
seg_dataset = "/home/nata-brain/Documents/proj/cv-train-fn/train/seg_labels"
sam_model_path = "/home/nata-brain/Documents/proj/cv-train-fn/sam2_l.pt"

yolo_bbox2segment(im_dir = train_dataset, save_dir = seg_dataset, sam_model = sam_model_path)

In [20]:
!pip install roboflow --quiet

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import numpy as np
import os
from tqdm import tqdm
import ultralytics
from ultralytics import YOLO
from ultralytics import SAM
from ultralytics.data import YOLODataset
from ultralytics.utils import LOGGER
from ultralytics.utils.ops import xywh2xyxy
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import os
import wandb
import timeit
from wandb.integration.ultralytics import add_wandb_callback

/home/nata-brain/miniconda3/envs/mpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nata-vito. Use `wandb login --relogin` to force relogin


True

In [7]:
dataset = "../datasets/mpt-cable-sm-dataset/data.yaml"

### Training Model

In [9]:
model_seg = YOLO("../models/cable-seg-sm.pt")

# Adding log to wandb project
add_wandb_callback(model_seg, enable_model_checkpointing=True)

model_seg.train(project = "cable-seg-sm", data = dataset, epochs = 100, imgsz = 320)

# Finish the W&B run
wandb.finish()

New https://pypi.org/project/ultralytics/8.3.13 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.79 🚀 Python-3.9.18 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24145MiB)
engine/trainer: task=segment, mode=train, model=../models/cable-seg-sm.pt, data=../datasets/mpt-cable-sm-dataset/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=cable-seg-sm, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=N

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


100%|██████████| 6.25M/6.25M [00:00<00:00, 62.4MB/s]


AMP: checks passed ✅


/home/nata-brain/miniconda3/envs/mpt/lib/python3.9/site-packages/ultralytics/engine/trainer.py:271: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/train/labels... 5216 images, 0 backgrounds, 401 corrupt: 100%|██████████| 5617/5617 [00:01<00:00, 3197.76it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 70/70 [00:06<00:00, 10.34it/s]


                   all       2236       2282      0.997      0.991      0.995      0.966      0.997      0.982      0.993      0.881


Generating Visualizations for batch-1/1: 100%|██████████| 32/32 [00:00<00:00, 45.09it/s]

Speed: 0.0ms preprocess, 1.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to cable-seg-sm/train4


lr/pg0,▃▆███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
lr/pg1,▆███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
lr/pg2,▆████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁
metrics/mAP50(B),▆▁▅▇▇▇███▇██████████████████████████████
metrics/mAP50(M),▁▂▅▄▄▇▇▇▇▂▅▅▇███▇█▆█▄▅▄▅▇████▆▆█▃▇▅█▅▄▄▆
metrics/mAP50-95(B),▂▁▃▄▄▅▄▅▆▆▅▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇█▇███▇▇█████
metrics/mAP50-95(M),▁▄▄▄▃▆▅▇▅▆▇▆▆▇▇▆▇▇▇▆█▆▆▅▆▇▇▇▆▇▇▇▇▇▆▇▇█▇█
metrics/precision(B),▄▂▁▇▇▇▇█▆███████▇▇█▇▇█▇▇▇███▇▇▇▇▇▇█▇▇▇▇▇
metrics/precision(M),▅▅▇▅▇█▁▅▅▇▇▅▅▇▃▅▃▆▇▄██▇▅▆▇▆▄▆▄▅▅▆▆█▅▃▅▇▅
metrics/recall(B),▇▅▁▇▇▇▇▇▇▇▇▇█▇▇█▇███▇███▇████▇██████████
metrics/recall(M),▆▁▅▆▆▆▇▇▇▇▇█▇▇█▇▇▇▇██▇▇▇▇██▇█▇▇▇▇▇▇▇▇▇▇▇


### Exporting model

In [ ]:
model_seg.export(format="onnx", optimize = True)

In [ ]:
model_seg.export(format="onnx")

In [10]:
model_to_val = YOLO("../models/cable-seg-7k.pt")

# Validate the model
metrics = model_to_val.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps  # a list contains map50-95(B) of each category
metrics.seg.map  # map50-95(M)
metrics.seg.map50  # map50(M)
metrics.seg.map75  # map75(M)
metrics.seg.maps  # a list contains map50-95(M) of each category

Ultralytics YOLOv8.2.79 🚀 Python-3.9.18 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24145MiB)
YOLOv8m-seg summary (fused): 245 layers, 27,222,963 parameters, 0 gradients, 110.0 GFLOPs


val: Scanning /home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/valid/labels.cache... 2236 images, 0 backgrounds, 20 corrupt: 100%|██████████| 2256/2256 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/valid/images/4011.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/valid/images/5311.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/valid/images/f1087dd7-107.jpg: ignoring corrupt image/label: cannot identify image file '/home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/valid/images/f1087dd7-107.jpg'
val: WARNING ⚠️ /home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/valid/images/f26e344c-303.jpg: ignoring corrupt image/label: cannot identify image file '/home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/valid/images/f26e344c-303.jpg'
val: WARNING ⚠️ /home/nata-brain/Documents/proj/image-enhancer/datasets/mpt-cable-sm-dataset/valid/images/f30dc1f1-324.jpg: ignoring c


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 140/140 [00:09<00:00, 15.37it/s]


                   all       2236       2282      0.997      0.991      0.995      0.965      0.997      0.982      0.992      0.875
Speed: 0.0ms preprocess, 2.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/segment/val


array([    0.87544])

### Inference

In [1]:
import numpy as np
from PIL import Image
import torch
import cv2 as cv

from ultralytics import YOLO
import matplotlib.pyplot as plt
import os
from pathlib import Path
import random
import albumentations as A
import timeit

model = YOLO("../models/cable-seg-7k.pt")

In [ ]:
print(type(model))

### Inference for segmentation

In [10]:
def segmentation_inference(model: YOLO, image: np.array, isIsolated: bool) -> np.array:
    if image is None:
        raise ValueError("Image is None, cannot process.")
    
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    results = model.predict(image, conf=0.8)
    isolated =  np.copy(image)
    
    for r in results:
        img = np.copy(r.orig_img)
        img_name = Path(r.path).stem  # source image base-name

        # Iterate each object contour (multiple detections)
        for ci, c in enumerate(r):
            label = c.names[c.boxes.cls.tolist().pop()]
            b_mask = np.zeros(img.shape[:2], np.uint8)

            # Create contour mask 
            contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2)
            _ = cv.drawContours(b_mask, [contour], -1, (255, 255, 255), cv.FILLED)

            if isIsolated:
                # Isolate object with black background
                mask3ch = cv.cvtColor(b_mask, cv.COLOR_GRAY2BGR)
                isolated = cv.bitwise_and(mask3ch, img)
            else:
                # Detection crop (from either OPT1 or OPT2)
                x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32)
                isolated = isolated[y1:y2, x1:x2]

    return isolated

def run_inference_on_batch(model: YOLO, image_data: np.array) -> np.array:
        start_time = timeit.default_timer()
        
        cropped_image = segmentation_inference(model, image_data, True)
        elapsed_time = timeit.default_timer() - start_time
        
        print(f'\n\n\nElapsed time: {elapsed_time:.2f} seconds')
       
        return cropped_image

In [16]:
dest = "../datasets/cable_dataset_tester/images/original"
output = "../datasets/cable_dataset_tester/images/dae_dataset"
os.makedirs(output, exist_ok = True)
i = 0

for file in os.listdir(dest):
    source = f"{dest}/{file}"
    
    img = cv.imread(source)
    image = apply_brightness_contrast(img, 100, 80)
    image = hist_equalization(image)
    image = clahe_cv(image)
    image = resize_img(image, 600, 300)
    image = run_inference_on_batch(model, image)
    cv.imshow("crop", image)
    
    cv.imwrite(f"{output}/{i}.jpg", image)
    
    if cv.waitKey(1) == ord('q'):
        break
    
    i += 1
   
            
""" 
results = model.predict('./mpt_cable_sm_dataset/train/images', conf=0.8)


with open(output_folder/) as f: 

for r in results:
    boxes = r.boxes
    masks = r.masks
    probs = r.probs
    print(f"Boxes: {boxes}, Masks: {masks}, Probs: {probs}")
"""



0: 160x320 1 cable, 3.6ms
Speed: 5.6ms preprocess, 3.6ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 320)



Elapsed time: 0.01 seconds

0: 160x320 1 cable, 3.8ms
Speed: 0.5ms preprocess, 3.8ms inference, 1.2ms postprocess per image at shape (1, 3, 160, 320)



Elapsed time: 0.01 seconds

0: 160x320 1 cable, 3.9ms
Speed: 0.5ms preprocess, 3.9ms inference, 1.5ms postprocess per image at shape (1, 3, 160, 320)



Elapsed time: 0.01 seconds

0: 160x320 1 cable, 3.9ms
Speed: 0.5ms preprocess, 3.9ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 320)



Elapsed time: 0.01 seconds

0: 160x320 1 cable, 5.0ms
Speed: 0.5ms preprocess, 5.0ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 320)



Elapsed time: 0.01 seconds

0: 160x320 1 cable, 3.9ms
Speed: 0.6ms preprocess, 3.9ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 320)



Elapsed time: 0.01 seconds

0: 160x320 1 cable, 3.8ms
Speed: 0.6ms preprocess, 3.8ms inference, 1.1ms postpro

' \nresults = model.predict(\'./mpt_cable_sm_dataset/train/images\', conf=0.8)\n\n\nwith open(output_folder/) as f: \n\nfor r in results:\n    boxes = r.boxes\n    masks = r.masks\n    probs = r.probs\n    print(f"Boxes: {boxes}, Masks: {masks}, Probs: {probs}")\n'

In [ ]:
from pathlib import Path
import os

def rebase_files(dir):
    i = 0 
    for file in os.listdir(dir):
        dest = f"./cable_dataset/train/images/{i}.jpg" 
        source = f"{dir}/{file}"
        os.rename(source, dest)
        i += 1
        
i = 7451
source = "../datasets/cable_dataset/train"
output = "../datasets/cable_dataset/images/dae_dataset"

for file in os.listdir(source):
    source = f"{source}/{file}"
    
    isTxt = Path(f'{source}').suffix == '.txt'
    print(source, isTxt)
    

   

In [ ]:
gray_img = cv2.imread("/home/nata-brain/Documents/proj/cv-train-fn/datasets/mpt_cable_sm_dataset/valid/images/ffd0ebe6-99.jpg")

seg_image =  run_inference_on_batch(gray_img)

cv.imwrite("test.jpg", seg_image)

## Semi-Supervised Learning

### Image preprocessing and Augmentation

In [14]:
import numpy as np
from PIL import Image
import torch
import cv2 as cv

from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

import random
import albumentations as A

def rename_files(dir):
    i = 0 
    for file in os.listdir(dir):
        dest = f"./cable_dataset/train/images/{i}.jpg" 
        source = f"{dir}/{file}"
        os.rename(source, dest)
        i += 1
        
def show_images(original, processed = False, isHist = False):

    fig = plt.figure(figsize=(10, 9)) 

    fig.add_subplot(1, 2, 1) 
    plt.imshow(original, cmap='gray') 
    plt.title("Original") 
    plt.show()
    
    fig.add_subplot(1, 2, 2) 
    if isHist:
        hist, bins = np.histogram(original.flatten(), 256, [0, 256])
        cdf = hist.cumsum()
        cdf_normalized = cdf * float(hist.max()) / cdf.max()

        plt.plot(cdf_normalized, color = 'b')
        plt.hist(original.flatten(), 256, [0, 256], color = 'r')
        plt.xlim([0, 256])
        plt.legend(('cdf', 'histogram'), loc = 'upper left')
    if processed:
        plt.imshow(processed) 
        plt.title("Processed") 

    plt.show()
    
def hist_equalization_cdf(image):
    hist, bins = np.histogram(image.flatten(), 256, [0, 256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()

    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m,0).astype('uint8')
    
    return cdf[image]

def hist_equalization(image):
    img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    dst = cv.equalizeHist(img)
    
    return dst

def resize_img(image, w, h):
    dim = (w, h)
    resized = cv.resize(image, dim, interpolation = cv.INTER_AREA)
    
    return resized

def binarize_image_CV(image):
    im_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    th, im_gray_th_otsu = cv2.threshold(im_gray, 128, 192, cv.THRESH_OTSU)
    
    return im_gray_th_otsu

def binarize_image_NP(image, thresh):
    im_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    #im_gray = np.array(Image.open(f'{path}').convert('L'))
    im_bin_keep = (im_gray > thresh) * im_gray
    
    return im_bin_keep

def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf
    
def dilatation(image, it, k):
    kernel = np.ones((k, k), np.uint8)
    dilated_im = cv.dilate(image, kernel, anchor=(0,0), iterations = it)
    
    return dilated_im

def clahe_cv(image):
    clahe = cv.createCLAHE(clipLimit = 20)
    gray_img1_clahe = clahe.apply(image)
    
    return gray_img1_clahe

        
def augmentation(image, seed_val):
    transform = A.Compose([
        A.RandomRotate90(),
        A.Flip(),
        A.Transpose(),
        A.GaussNoise(),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.RandomBrightnessContrast(),
        ], p=0.3)
    ])
    
    random.seed(seed_val)
    transformed_image = transform(image=image)['image']
    
    return transformed_image


def preprocessing_image(dir):
    dest = "./cable_dataset/train/images/pr"
    i = 0 
    seed = 0
    
    os.makedirs(dest, exist_ok = True)

    for file in os.listdir(dir):
        dest_img = f"{dest}/{i}.jpg" 
        source = f"{dir}/{file}"
        image = cv.imread(source)
        image = resize_img(image, 320, 320)
        image = apply_brightness_contrast(image, 100, 80)
        image = hist_equalization(image)
        image = clahe_cv(image)
        image = augmentation(image, seed)
        #cv.imshow("Img", image)
        cv.imwrite(dest_img, image)
        
        if cv.waitKey(1) == ord('q'):
            break
        
        if seed == 100:
            seed = 0
        
        
        i += 1
        seed += 1

In [ ]:
dataset = "./datasets/cable_dataset/train/images/original/"
original_image_pil = f"{dataset}/1.jpg"
original_image_cv = cv.imread(f"{dataset}1.jpg")
#show_images(original_image_cv, False, True)
     
preprocessing_image(dataset)

### Image processing pipeline

In [ ]:

img4 = resize_img(original_image_cv)
show_images(img4, False, False)

cv.imwrite("img4.jpg", img4)

contrast_img = apply_brightness_contrast(img4, 80, 80)
contrast_img = hist_equalization(contrast_img)
clahe = cv.createCLAHE(clipLimit = 20)
gray_img1_clahe = clahe.apply(contrast_img)

transform = A.Compose([
        A.RandomRotate90(),
        A.Flip(),
        A.Transpose(),
        A.GaussNoise(),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.RandomBrightnessContrast(),
        ], p=0.3)
    ])
random.seed(42)
transformed_image_1 = transform(image=gray_img1_clahe)['image']
show_images(seg_image, False, False)



cv.imwrite("test.jpg", gray_img1_clahe)

In [15]:
rename_files("./cable_dataset/images")